한계점 - 폐과포함

```python
df = df[~df['학교명'].str.contains('대학원')]
df = df[['학부_과(전공)명', '학교명']]
```
->TypeError: bad operand type for unary ~: 'float'


해당 코드에서 발생하는 오류는 '~' 연산자가 부동 소수점(float) 타입에 적용되어 발생하는 타입 오류입니다. 이 오류는 데이터프레임의 '학교명' 열에 NaN(결측치)이 있어서 발생할 수 있습니다.

해결 방법으로는 '학교명' 열의 결측치를 먼저 처리한 후 '~' 연산자를 사용하여 조건을 만족하는 행을 선택할 수 있습니다.

```python
# '학교명' 열의 결측치를 제거한 후 조건을 만족하는 행 선택
df = df[~df['학교명'].fillna('').str.contains('대학원')]

# 필요한 열만 선택하여 데이터프레임 재구성
df = df[['학부_과(전공)명', '학교명']]

print(df)
```

위의 수정된 코드는 '학교명' 열의 결측치를 빈 문자열('')로 대체한 후에 '~' 연산자를 사용하여 '대학원'이 포함되지 않은 행을 선택합니다. 그리고 필요한 열만 선택하여 데이터프레임을 재구성하고 출력합니다.

```
f_grouped = df.groupby('학부_과(전공)명')['학교명'].apply(lambda x: ', '.join(x)).reset_index()
df_grouped.to_csv('C:/Users/황의지/Desktop/hk+/major_uni.csv', index=False, encoding='utf-8-sig') 
```
->TypeError: sequence item 0: expected str instance, float found
```python
df_grouped = df.groupby('학부_과(전공)명')['학교명'].apply(lambda x: ', '.join(x.dropna())).reset_index()
df_grouped.to_csv('C:/Users/황의지/Desktop/hk+/major_uni.csv', index=False, encoding='utf-8-sig')
```

위의 코드에서 변경된 부분은 `lambda x: ', '.join(x.dropna())`입니다. 이 부분은 결측치를 제거하고 문자열을 결합하기 위해 `dropna()` 함수를 사용하여 NaN 값을 제거하는 부분입니다.

# Method_1

In [1]:
import time
import requests
import pandas as pd
from xml.etree import ElementTree

#start = time.time()  # 시작 시간 저장

url = 'https://api.odcloud.kr/api/15014632/v1/uddi:6939f45b-1283-4462-b394-820c26e1445d'
params = {
    'page': 1,
    'perPage': 7000,
    'returnType': 'XML',
    'serviceKey': 'GjCdPonj23F6SdOIs8TsI4OEUkZIi3oa6PcTvdoGQZQIDMZTgCYzn4HlzpcPKxxxJXECIt+rqU1GNFtm3fB8IA=='
}

data = []

while True:
    response = requests.get(url, params=params)
    xml_data = response.text
    root = ElementTree.fromstring(xml_data)

    for item in root.findall('data/item'):
        item_data = {}
        for col in item.findall('col'):
            name = col.get('name')
            if name == '학교명' or name == '학부_과(전공)명':
                item_data[name] = col.text
        data.append(item_data)

    if int(root.findtext('totalCount')) <= len(data):
        break

    params['page'] += 1

df = pd.DataFrame(data)

df = df[~df['학교명'].fillna('').str.contains('대학원')]
df = df[['학부_과(전공)명', '학교명']]

print(df)

#print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

      학부_과(전공)명     학교명
0         기계공학과   조선대학교
1         기계공학과   한밭대학교
2         기계공학과   단국대학교
11        기계공학부  성균관대학교
12        기계공학부   영남대학교
...         ...     ...
52209       화학과   울산대학교
52210     물리학전공   울산대학교
52211      물리학과   울산대학교
52212    자연과학계열   울산대학교
52213  교양대학(진천)   우석대학교

[33043 rows x 2 columns]
time : 24.637258529663086


# Method_2

In [2]:
import time
import requests
import pandas as pd
from xml.etree import ElementTree

#start = time.time()

url = 'https://api.odcloud.kr/api/15014632/v1/uddi:6939f45b-1283-4462-b394-820c26e1445d'
params = {
    'page': 1,
    'perPage': 2906,
    'returnType': 'XML',
    'serviceKey': 'GjCdPonj23F6SdOIs8TsI4OEUkZIi3oa6PcTvdoGQZQIDMZTgCYzn4HlzpcPKxxxJXECIt+rqU1GNFtm3fB8IA=='
}

data = []

for page in range(1, 38): 
    params['page'] = page
    response = requests.get(url, params=params)
    xml_data = response.text
    root = ElementTree.fromstring(xml_data)

    for item in root.findall('data/item'):
        item_data = {}
        for col in item.findall('col'):
            name = col.get('name')
            if name == '학교명' or name == '학부_과(전공)명':
                item_data[name] = col.text
        data.append(item_data)

    if page == 55214:  # 원하는 페이지 수에 도달하면 반복문 종료
        break

df = pd.DataFrame(data)
'''
df = df[~df['학교명'].str.contains('대학원')]
df = df[['학부_과(전공)명', '학교명']]
'''
# '학교명' 열의 결측치를 제거한 후 조건을 만족하는 행 선택
df = df[~df['학교명'].fillna('').str.contains('대학원')]

# 필요한 열만 선택하여 데이터프레임 재구성
df = df[['학부_과(전공)명', '학교명']]

print(df)

#print("time :", time.time() - start)

      학부_과(전공)명     학교명
0         기계공학과   조선대학교
1         기계공학과   한밭대학교
2         기계공학과   단국대학교
11        기계공학부  성균관대학교
12        기계공학부   영남대학교
...         ...     ...
52209       화학과   울산대학교
52210     물리학전공   울산대학교
52211      물리학과   울산대학교
52212    자연과학계열   울산대학교
52213  교양대학(진천)   우석대학교

[33043 rows x 2 columns]
time : 32.302387952804565


In [16]:
df_grouped = df.groupby('학부_과(전공)명')['학교명'].apply(lambda x: ', '.join(x.dropna())).reset_index()
df_grouped.to_csv('C:/Users/황의지/Desktop/hk+/major_uni.csv', index=False, encoding='utf-8-sig')